In [5]:
#importing libraries
!pip install kaggle
import kaggle

In [6]:
#download data from kaggle 
!kaggle datasets download ankitbansal06/retail-orders

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 20.2MB/s]


In [8]:
# unzipping retail order 
import zipfile

# Path to the zip file
zip_path = "retail-orders.zip"

# Unzip in the current directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

print("✅ Extracted in the same directory.")



✅ Extracted in the same directory.


In [12]:
#read data from the file and handle null values
import pandas as pd 
df = pd.read_csv("orders.csv",na_values=[ 'Not Available', 'unknown' ])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [28]:
#rename columns by making them lower cases and also replacing he space with an underscore 
#df.columns=df.columns.str.lower()
#df.columns=df.columns.str.replace(' ', "_")
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [38]:
#Generating new columns like discount_price, sale_price and profit 
#df['discount'] = df['list_price']*df['discount_percent']/100
#df['sale_price'] = df['list_price']-df['discount']
df['profit'] = df['sale_price']-df['cost_price']

In [55]:
#converting order_date to date
# note initially the order type is in type object, therefore we will convert it to a datetime data type 
df['order_date'] = pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [56]:
#drop unnecessary column 
df.drop(columns = ['discount_percent', 'list_price', 'cost_price'], inplace = True)

KeyError: "['discount_percent', 'list_price', 'cost_price'] not found in axis"

In [62]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load variables from .env file
load_dotenv()

# Read MySQL credentials from environment variables
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
database = os.getenv('MYSQL_DB')

# Build SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

# Optional test: print engine URL (without password)
print(f"🔗 Connected to MySQL at {host}:{port}, database: {database}")


🔗 Connected to MySQL at localhost:3306, database: world


In [65]:
#loading the data into the database 

df.to_sql('orders', con=engine, index=False, if_exists='append')


9994

In [66]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')